# Redes de Proteção às Mulheres do DF - TJDFT

O intuito do script abaixo é para a extração dos dados da cartilha com informações sobre as redes de proteção presentes em cada região administrativa do Distrito Federal.

### Importando os dados e as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import PyPDF2 as pf
import re

Carregamento do arquivo PDF

In [2]:
pdf_file = pf.PdfFileReader('Dados/catalogo_DF.pdf')

Definição das RA's e as suas respectivas páginas do pdf

In [3]:
lista_RA = ['ÁGUAS CLARAS', 'ARNIQUEIRAS', 'ASA NORTE', 'ASA SUL',
            'BRAZLÂNDIA', 'CANDANGOLâNDIA', 'CEILÂNDIA', 'CRUZEIRO', 
            'DISTRITO FEDERAL', 'ESTRUTURAL / SCIA', 'FERCAL', 
            'GAMA', 'GRANJA DO TORTO', 'GUARÁ', 'ITAPOÃ', 'JARDIM BOTâNICO', 
            'LAGO NORTE', 'LAGO SUL', 'NÚCLEO BANDEIRANTE', 'PARANOÁ', 
            'PARK WAY', 'PLANALTINA', 'PÔR DO SOL / SOL NASCENTE', 'RECANTO DAS EMAS',
            'RIACHO FUNDO I', 'RIACHO FUNDO II', 'SAMAMBAIA', 'SANTA MARIA', 
            'SÃO SEBASTIÃO', 'SIA - SETOR DE INDÚSTRIA E ABASTECIMENTO', 'SOBRADINHO I', 
            'SOBRADINHO II', 'SUDOESTE / OCTOGONAL', 'TAGUATINGA', 'VARJÃO', 
            'VICENTE PIRES', 'VILA PLANALTO']
lista_indices = [9, 15, 15, 20, 21, 28, 28, 37, 38, 44, 44, 47, 48, 59, 60, 63, 64, 69, 69, 74, 74, 77, 78, 84, 84, 87, 87, 92, 93, 97, 97, 101, 101, 104, 105, 109, 109, 113, 114, 119, 120, 123, 123, 135, 136, 141, 142, 149, 150, 154, 155, 159, 159, 167, 168, 175, 176, 183, 184, 188, 189, 195, 196, 200, 200, 204, 205, 215, 215, 219, 219, 225, 225, 229]

### Definindo as funções

Função para limpar o texto bruto

In [4]:
def limpar_texto_bruto(text=''):
    lista_limpar = ['\n', 'Rede de Proteção às Mulheres do Distrito Federal | 3ª edição Núcleo Judiciário da Mulher - NJM/TJDFT', 'Internação Álcool / Drogas:  Para acessar as instituições de internação conveniadas à Secretaria de Saúde do Distrito Federal, o(a) jurisdicionado(a) deve ser encaminhado ao CAPS AD de referência do seu local de residência para avaliação multidisciplinar que, por sua vez, encaminha para a rede conveniada conforme critérios da equipe de saúde.', 'Programas Assistência Social:  Para acessar os programas da Assistência Social, o(a) jurisdicionado(a) deve ser encaminhado ao CRAS/ CREAS de referência do seu local de residência para avaliação multidisciplinar que, por sua vez, encaminha para a rede conveniada conforme critérios da equipe do Sistema Único de Assistência Social.', 'Programas Assistência Social:  Para acessar os programas da Assistência Social, o(a) jurisdicionado(a) deve ser encaminhado ao CRAS/ CREAS de referência do seu local de residência para avaliação multidisciplinar que, por sua vez, encaminha para a rede conveniada conforme critérios da equipe do Sistema Único de Assistência Social.','Rede de Proteção às Mulheres do Distrito Federal | 3ª edição Núcleo Judiciário da Mulher - NJM/TJDFT','Internação Álcool / Drogas:  Para acessar as instituições de internação conveniadas à Secretaria de Saúde do Distrito Federal, o(a) jurisdicionado(a) deve ser encaminhado ao CAPS AD de referência do seu local de residência para avaliação multidisciplinar que, por sua vez, encaminha para a rede conveniada conforme critérios da equipe de saúde.”','Programas Assistência Social:  Para acessar os programas da Assistência Social, o(a) jurisdicionado(a) deve ser encaminhado ao CRAS/ CREAS de referência do seu local de residência para avaliação multidisciplinar que, por sua vez, encaminha para a rede conveniada conforme critérios da equipe do Sistema Único de Assistência Social.”','Internação Álcool / Drogas:  Para acessar as instituições de internação conveniadas à Secretaria de Saúde do Distrito Federal, o(a) jurisdicionado(a) deve ser encaminhado ao CAPS AD de referência do seu local de residência para avaliação multidisciplinar que, por sua vez, encaminha para a rede conveniada conforme critérios da equipe de saúde.”','Programas Assistência Social:  Para acessar os programas da Assistência Social, o(a) jurisdicionado(a) deve ser encaminhado ao CRAS/ CREAS de referência do seu local de residência para avaliação multidisciplinar que, por sua vez, encaminha para a rede conveniada conforme critérios da equipe do Sistema Único de Assistência Social.”','Rede de Proteção às Mulheres do Distrito Federal | 3ª edição Núcleo Judiciário da Mulher - NJM/TJDFT 17 16ÁGUAS CLARAS A A ÁGUAS CLARAS','Rede de Proteção às Mulheres do Distrito Federal | 3ª edição Núcleo Judiciário da Mulher - NJM/TJDFT 27 26ÁGUAS CLARAS ÁGUAS CLARASA A','Internação Álcool / Drogas: Para acessar as instituições de internação conveniadas à Secretaria de Saúde do Distrito Federal, o(a) jurisdicionado(a) deve ser encaminhado ao CAPS AD de referência do seu local de residência para avaliação multidisciplinar que, por sua vez, encaminha para a rede conveniada conforme critérios da equipe de saúde.','Programas Assistência Social: Para acessar os programas da Assistência Social, o(a) jurisdicionado(a) deve ser encaminhado ao CRAS/ CREAS de referência do seu local de residência para avaliação multidisciplinar que, por sua vez, encaminha para a rede conveniada conforme critérios da equipe do Sistema Único de Assistência Social.','Programas Assistência Social:  Para acessar os programas da Assistência Social, o(a) jurisdicionado(a) deve ser encaminhado ao CRAS/ CREAS de referência do seu local de residência para avaliação multidisciplinar que, por sua vez, encaminha para a rede conveniada conforme critérios da equipe do Sistema Único de Assistência Social.','Rede de Proteção às Mulheres do Distrito Federal | 3ª edição Núcleo Judiciário da Mulher - NJM/TJDFT 21 20ÁGUAS CLARAS ÁGUAS CLARASA A','Rede de Proteção às Mulheres do Distrito Federal | 3ª edição Núcleo Judiciário da Mulher - NJM/TJDFT 19 18ÁGUAS CLARAS ÁGUAS CLARASA A']
    for c in lista_limpar:
        text = text.replace(c, '')
    return text

Função para gerar um dicionário com cada informação de uma instituição

In [23]:
def get_dic_v2(t1):
    categorias = ['Endereço', 'Telefones Gerais', 'Fax', 'E-mail', 'Observações', 'Observação', 'Atendimento', 'Horário', 'Público', 'Bens e Serviços', 'Bens e serviços', 'Critérios', 'Critério']
    l = []

    # Busca os índices de todos os tipos de informações
    for c in categorias:
        if c == 'Atendimento':
            l.append(t1.find(c, 30))
        else:
            l.append(t1.find(c))

    # Dicionário apenas com os tipos de informações presentes
    dic_contem = {}
    for c in range(len(l)):
        if l[c] != -1:
            dic_contem[categorias[c]] = l[c]
    dic_contem = dict(sorted(dic_contem.items(), key=lambda item: item[1]))

    # Coleta as informações de cada tipo de informações e transforma em um dicionário
    cont, dic_final= 0, {}
    nome, indice = '', 0
    for key, value in dic_contem.items():
        if cont != 0:
            dic_final[nome] = [t1[indice + len(nome):value]]
        else:
            dic_final['Instituição'] = [t1[:value]]
            indice = value
        cont = 1
        nome = key
        indice = value
    dic_final[nome] = [t1[indice + len(nome):]]
    return dic_final

Função para gerar um dataset de uma RA dada

In [24]:
def get_dataset(nome_ra, texto_tratado):
    tipos_info = ['ADMINISTRAÇÃO', 'ÁLCOOL, DROGAS', 'ASSISTÊNCIA SOCIAL', 'DIREITO, JUSTIÇA, SEGURANÇA PÚBLICA', 'EDUCAÇÃO', 'ESPORTE/ LAZER/ CULTURA', 'PESQUISA/CONSULTORIA', 'PREVIDÊNCIA SOCIAL', 'SAÚDE, SAÚDE MENTAL', 'SAÚDE/SAÚDE MENTAL','TRABALHO, FORMAÇÃO PROFISSIONAL','TRABALHO/ FORMAÇÃO PROFISSIONAL','VIOLÊNCIA']
    text_adm, text_alcool, text_assistencia, text_djs, text_edu, text_esporte, text_pesquisa, text_previdencia, text_saude, text_trabalho, text_violencia = '', '', '', '', '', '', '', '', '', '', ''
    lista_textos = [text_adm, text_alcool, text_assistencia, text_djs, text_edu, text_esporte, text_pesquisa, text_previdencia, text_saude, text_trabalho, text_violencia]
    indi_tipos, nome_contem, ind_contem = [], [], []
    for tipo in tipos_info:
        indi_tipos.append(texto_tratado.find(tipo))
    
    c_indice = 0
    first = True
    for indice in indi_tipos:
        if indice > -1:
            nome_contem.append(tipos_info[c_indice])
            ind_contem.append(indice)
        c_indice += 1

    # ['ADMINISTRAÇÃO', 'ÁLCOOL, DROGAS', 'ASSISTÊNCIA SOCIAL', 'DIREITO, JUSTIÇA, SEGURANÇA PÚBLICA', 'SAÚDE/SAÚDE MENTAL', 'VIOLÊNCIA']
    # [32, 237, 1966, 4983, 10270, 11995]
    lista_textos = []
    for k, v in enumerate(nome_contem):
        i = ind_contem[k]
        # k -> contador
        # v -> nome da categoria
        len_cat = len(v)
        # i -> indice da categoria
        if k < 1:
            ind_ant = 0
        if k < len(nome_contem) - 1:
            text_cat = texto_tratado[ind_contem[k] + len_cat - 1: ind_contem[k + 1]].replace(v, '')
        else:
            text_cat = texto_tratado[ind_contem[k] + len_cat - 1:].replace(v, '')
        lista_textos.append(text_cat)
    
    data_teste = pd.DataFrame()
    contador = 0
    for texto in lista_textos:
        for trecho in texto.split('Instituição'):
            if trecho != '':
                dic_tratado = get_dic_v2(trecho)
                data_temp = pd.DataFrame(dic_tratado)
                data_temp['Tipo'] = nome_contem[contador]
                data_teste = pd.concat([data_teste, data_temp], ignore_index=True, join='outer')
        contador += 1
    data_teste_sb = data_teste

    data_teste['Região Administrativa'] = nome_ra
    data_teste = data_teste[['Região Administrativa','Tipo','Instituição', 'Endereço', 'Telefones Gerais', 'E-mail', 'Observações',
           'Atendimento', 'Horário', 'Público', 'Critérios']]

    data_teste_an = data_teste
    return data_teste_an

### Extração dos dados

In [25]:
cont = 0
data_all = pd.DataFrame()
for k, v in enumerate(lista_RA):
    nome_ra = v
    # k valor de início e valor final
    texto_bruto = ''
    for c in range(lista_indices[cont], lista_indices[cont + 1]):
        tg = pdf_file.pages[c].extract_text()
        texto_bruto = ''.join([texto_bruto, tg])
    cont += 2
    texto_tratado = limpar_texto_bruto(texto_bruto)
    
    data_temp = get_dataset(nome_ra, texto_tratado)
    data_all = pd.concat([data_all, data_temp], ignore_index=True, join='outer')

In [26]:
data_all.shape

(1522, 11)

### Tratamento dos dados

In [31]:
data = data_all.dropna(subset=['Instituição']).reset_index(drop=True)
data

,Região Administrativa,Tipo,Instituição,Endereço,Telefones Gerais,E-mail,Observações,Atendimento,Horário,Público,Critérios
0,ÁGUAS CLARAS,ADMINISTRAÇÃO,Administração Regional de Águas Claras,"Rua Manacá Lote 2 Bloco 1, Águas Claras",3383-8929,atendimento.raxx@gmail.com,NaN,De segunda a sexta-feira,Das 8h às 11h e das 14h às 18h,Todos,NaN
1,ÁGUAS CLARAS,"ÁLCOOL, DROGAS",Alcoólicos Anônimos (AA) – Grupo Águas Claras,"R. das Figueiras, lotes 2/4 - Q. 101, s/n Ed. ...",3226-0091 / 3351-9644 (escritórios em Brasília),NaN,Para ser membro de A.A. não há taxas ou mensa...,De segunda a sexta-feira,Das 20h às 22h,Adultos (homens e mulheres) que possuem proble...,
2,ÁGUAS CLARAS,"ÁLCOOL, DROGAS",Al-Anon – Grupo Águas Claras,SWPW Quadra 101 Rua das Figueiras Residencial ...,NaN,NaN,Para ser membro de Al-Anon não há taxas ou me...,Segundas-feiras,Das 20h às 22h,Familiares e amigos de alcoolistas. Abrange ta...,NaN
3,ÁGUAS CLARAS,"ÁLCOOL, DROGAS",Narcóticos Anônimos -NA,"Rua das Figueiras, Lotes 2/4, Ed. Figueiras Sa...",NaN,NaN,NaN,Quinta-feira,NaN,NaN,NaN
4,ÁGUAS CLARAS,"ÁLCOOL, DROGAS",Alcoólicos Anônimos (AA) – Grupo Águas Claras,NaN,NaN,NaN,O NA é uma irmandade ou sociedade sem fins luc...,NaN,20h,"Adultos, homens e mulheres que possuem problem...",NaN
...,...,...,...,...,...,...,...,...,...,...,...
1252,VILA PLANALTO,SAÚDE/SAÚDE MENTAL,CAPSi (Centro de Atenção Psicossocial Infanto-...,"SMHN, Qd. 03, Conj. 1, Bloco A, Ed. COMPP – As...",2017-1900 (Ramais 7710 e 7711),capsi.asanorte@yahoo.com.br,NaN,Segunda a sexta.,8h – 17h.,Infanto-Juvenil.,
1253,VILA PLANALTO,SAÚDE/SAÚDE MENTAL,Unidade Básica de Saúde Nº. 03 – Vila Planalto,"Rua Piauí, área especial s/nº – Vila Planalto.",NaN,csb15vilaplanalto@gmail.com,NaN,ambulatorial e atenção básica preventiva.,Segunda a sexta-feira. das 07:00 às 18:00 e ao...,Todos.,NaN
1254,VILA PLANALTO,VIOLÊNCIA,Pró-Vítima - Subsecretaria de Proteção às Vít...,"Estação Rodoferroviária, Ala Central, Térreo,...",2104-4289,provitima.sejusgdf@gmail.com,NaN,De segunda a sexta-feira,8h às 18h,Todos,
1255,VILA PLANALTO,VIOLÊNCIA,Centro de Especialidades para a Atenção às Pes...,temporário (atualizado em maio/21) COMPP - SM...,2017-1900 / R: 7169 / Whatsapp (61) 992370336 ...,programamargarida.hran@gmail.com.br,Acolhimento -,"individual (psicossocial e médico), familiar ...",Das 8h às 18h.,"Pessoas acima de 18 anos, adultos em situação ...",NaN


### Exportação dos dados

In [30]:
data.to_excel('cartilha_dados.xlsx', index=False)